In [23]:
import boto3
from sagemaker import get_execution_role

sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

account_id = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name

#used to store model artifacts which SageMaker will extract to /opt/ml/model in the container, 
#in this example case we will not be making use of S3 to store the model artifacts
#s3_bucket = '<S3Bucket>'

role = get_execution_role()
role

'arn:aws:iam::584910123950:role/AmazonSageMaker-ExecutionRole'

In [31]:
from time import gmtime, strftime

model_name = 'spacy-nermodel-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
# MODEL S3 URL containing model atrifacts as either model.tar.gz or extracted artifacts. 
# Here we are not  
#model_url = 's3://{}/spacy/'.format(s3_bucket) 

container = '{}.dkr.ecr.{}.amazonaws.com/sm-pretrained-spacy:latest'.format(account_id, region) 
instance_type = 'ml.m5.24xlarge'

print('Model name: ' + model_name)
#print('Model data Url: ' + model_url)
print('Container image: ' + container)

container = {
'Image': container
}

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    Containers = [container])

print("Model Arn: " + create_model_response['ModelArn'])

Model name: spacy-nermodel-2024-09-09-19-47-45
Container image: 584910123950.dkr.ecr.us-east-1.amazonaws.com/sm-pretrained-spacy:latest
Model Arn: arn:aws:sagemaker:us-east-1:584910123950:model/spacy-nermodel-2024-09-09-19-47-45


In [32]:
endpoint_config_name = 'spacy-ner-config' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': instance_type,
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName': model_name,
        'VariantName': 'AllTraffic'}])
        
print("Endpoint config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

Endpoint config name: spacy-ner-config2024-09-09-19-47-46
Endpoint config Arn: arn:aws:sagemaker:us-east-1:584910123950:endpoint-config/spacy-ner-config2024-09-09-19-47-46


In [33]:
%%time

import time

#endpoint_name = 'flan-t5-endpoint' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
endpoint_name = 'spacy-ner-endpoint' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint name: ' + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)


Endpoint name: spacy-ner-endpoint2024-09-09-19-47-50
Endpoint Arn: arn:aws:sagemaker:us-east-1:584910123950:endpoint/spacy-ner-endpoint2024-09-09-19-47-50
Endpoint Status: Creating
Waiting for spacy-ner-endpoint2024-09-09-19-47-50 endpoint to be in service...
CPU times: user 35.4 ms, sys: 1.13 ms, total: 36.5 ms
Wall time: 2min 31s


In [34]:
import json
content_type = "application/json"
request_body = {"input": "This is a test with NER in America with \
    Amazon and Microsoft in Seattle, writing random stuff."}

#Serialize data for endpoint
#data = json.loads(json.dumps(request_body))
payload = json.dumps(request_body)

#Endpoint invocation
response = runtime_sm_client.invoke_endpoint(
EndpointName=endpoint_name,
ContentType=content_type,
Body=payload)

#Parse results
result = json.loads(response['Body'].read().decode())['output']
result

[['NER', 'ORG'],
 ['America', 'GPE'],
 ['Amazon', 'ORG'],
 ['Microsoft', 'ORG'],
 ['Seattle', 'GPE']]

In [6]:
sm_client.delete_endpoint(EndpointName=endpoint_name)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)

{'ResponseMetadata': {'RequestId': '679db93e-751c-4d23-9509-dc451dcda962',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '679db93e-751c-4d23-9509-dc451dcda962',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 09 Sep 2024 17:19:29 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}